In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd

In [2]:
import re

def clean_elements(array):
    cleaned_array = []
    for element in array:
        # Use regular expression to remove (GO:xxx) substring,  R-HSA-xxx substrings, and WPxxx substrings
        cleaned_element = re.sub(r'\s*\(GO:\d+\)\s*|\s*R-HSA-\d+\s*|\s*WP\d+\s*', '', element)
        cleaned_array.append(cleaned_element)
    return cleaned_array

In [3]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human", 
             "GO_Biological_Process_2023",
             "GO_Molecular_Function_2023",
             "GO_Cellular_Component_2023"]
#lib_names = ["KEGG_2021_Human"]
#lib_names = ["KEGG_2021_Human", 
#             "Reactome_2022", 
#             "WikiPathway_2023_Human"]

In [4]:
# Generate table for some basic stats on gene set libraries used.
geneset_sz = []
for lib_name in lib_names:
    (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
    x = [{"lib_name": lib_name, "descr": d, "num_genes": len(g)} for (d,g) in zip(descr,genes)]
    print(len(genes))
    geneset_sz.extend(x)
df = pd.DataFrame(geneset_sz)
df.to_csv("geneset_sz.tsv",sep="\t",index=None)

320
1818
801
5407
1147
474


In [5]:
aclient = openai.AsyncClient()
models = ["gpt-3.5-turbo-0125", "gpt-4o-2024-05-13"]
# "gpt-4-turbo-2024-04-09" has garbage random text, doesn't generat correctly using
# the async API
# models = ["gpt-4-turbo-2024-04-09"]
for model in models:
    for lib_name in lib_names:
        (descr, genes) = llm2geneset.read_gmt("libs_human/gmt/" + lib_name + ".txt")
        # Generate cleaned version of gene set description w/o identifiers.
        descr_cleaned = clean_elements(descr)
        
        # Generate genes, and keep track of time it takes (seconds).
        start_time = time.time()
        llm_genes = await llm2geneset.get_genes(aclient, 
                                                descr_cleaned, 
                                                model=model, use_sysmsg=True)
        end_time = time.time()
        gen_time = end_time - start_time

        llm_genes_norole = await llm2geneset.get_genes(aclient, descr_cleaned, 
                                                       model=model, use_sysmsg=False)

        gen_res = {}

        # Assemble and save generation results.
        gen_res["lib_name"] = lib_name
        gen_res["model"] = model
        gen_res["gen_time"] = gen_time
        gen_res["descr"] = descr
        gen_res["descr_cleaned"] = descr_cleaned
        gen_res["curated_genesets"] = genes
        gen_res["llm_genesets"] = llm_genes
        gen_res["llm_genes_norole"] = llm_genes_norole
    
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)

 86%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 274/320 [00:06<00:01, 25.24it/s]

retrying
'gene'
List all the known genes directly and indirectly involved in the following biological process or cellular component """Ubiquitin mediated proteolysis""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Ubiquitin mediated proteolysis""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {
        "gene": "UBE2D1"
    },
    {
        "gene": "UBE2D2"
    },
    {
        "gene": "UBE2L3"
    },
    {
        "gene": "UBE2N"
    },
    {
        "gene": "UBE2V1"
    },
    {
        "gene": "UBE2W"
    },
    {
        "gene": "CDC34"
    },
    {
        "gene

 49%|████████████████████████████████████████████████████████████▊                                                               | 891/1818 [00:08<00:06, 140.69it/s]

retrying
list indices must be integers or slices, not str
List all the known genes directly and indirectly involved in the following biological process or cellular component """Platelet Homeostasis""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Platelet Homeostasis""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {"gene": "ITGA2B"},
    {"gene": "ITGB3"},
    {"gene": "GP1BA"},
    {"gene": "GP1BB"},
    {"gene": "GP5"},
    {"gene": "GP6"},
    {"gene": "GP9"},
    {"gene": "GPX1"},
    {"gene": "GPX3"},
    {"gene": "VWF"},
    {"gene": "thromboxane A2 receptor (

 61%|██████████████████████████████████████████████████████████████████████████▋                                                | 1104/1818 [00:09<00:04, 148.20it/s]

...

  9%|██████████▉                                                                                                                  | 474/5407 [00:08<00:50, 96.90it/s]

retrying
'gene'
List all the known genes directly and indirectly involved in the following biological process or cellular component """Phosphatidylcholine Metabolic Process""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """Phosphatidylcholine Metabolic Process""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {
        "gene": "CHPT1"
    },
    {
        "gene": "CHKA"
    },
    {
        "gene": "CHKB"
    },
    {
        "gene": "PCYT1A"
    },
    {
        "gene": "PCYT1B"
    },
    {
        "gene": "CEPT1"
    },
    {
        "gene": "LPCAT1"
    },
    {
  

 31%|█████████████████████████████████████▋                                                                                     | 1655/5407 [00:17<00:27, 134.01it/s]

retrying
'gene'
List all the known genes directly and indirectly involved in the following biological process or cellular component """protein-RNA Complex Disassembly""". Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "gene": {
                "type": "string",
            }
        },
        "required": ["gene"]
    }
}
```
The field `gene` is a gene involved in the following biological process or cellular component: """protein-RNA Complex Disassembly""". Use the HUGO Gene Nomenclature Committee (HGNC) gene abbreviations. Place the output in a JSON code block. Do not add any comments in the JSON code block.

```json
[
    {
        "gene": "DDX5"
    },
    {
        "gene": "DDX6"
    },
    {
        "Gene": "DHX9"
    },
    {
        "gene": "EXOSC10"
    },
    {
        "gene": "MOV10"
    },
    {
        "gene": "UPF1"
    },
    {
        "gene": "XRN1"
    },
    {
        "gene": "H

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 474/474 [00:22<00:00, 20.87it/s]
